# Part III. Classes and Protocols

# Chapter 11. A Pythonic Object

"Để một thư viện hoặc framework trở thành Pythonic, nghĩa là làm cho lập trình viên Python nắm bắt cách thực hiện một tác vụ dễ dàng và tự nhiên nhất có thể." - như cách mà Python cung cấp

In this chapter, we will see how to:

- Support the built-in functions that convert objects to other types (e.g., `repr()`, `bytes()`, `complex()`, etc.)

- Implement an alternative constructor as a class method

- Extend the format mini-language used by `f-strings`, the `format()` built-in, and the `str.format()` method

- Provide read-only access to attributes

- Make an object hashable for use in sets and as dict keys

- Save memory with the use of `__slots__`

Quay lại ví dụ `Vector2d`
- Chap 12 nói về Vector N chiều
- Nói về khi nào dung `@classmethod` và `@staticmethod`
- Private và protected attributes

## Object Representations
string representation of object
Python có 2 loại:
- `repr()`/`__repr__`: string mà dev muốn thấy
- `str()`/`__str__`: string mà user muốn thấy
    - `__bytes__`: byte sequence
    - `__format__`: `str.format()`



## Vector Class Redux


In [1]:
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4> v1 --> Vector2d(3.0, 4.0)

    def __str__(self):
        return str(tuple(self))  # <5> print(v1) --> (3.0, 4.0)

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>

#1: converting Vector2d instances to/from bytes.
#3: __iter__ unpacking work (e.g, x, y = my_vector)
#4: __repr__, {!r} to get x, y

In [2]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [3]:
x, y = v1
x, y

(3.0, 4.0)

In [4]:
v1

Vector2d(3.0, 4.0)

In [5]:
# eval chỉ rằng repr của Vector2d là một biểu diễn của lệnh gọi hàm tạo
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [6]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [7]:
abs(v1)

5.0

In [8]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

Bug ở version 0 này

In [10]:
Vector2d(3, 4) == [3, 4]

True

Chương 16 handle nó bằng operator overloading.

## An Alternative Constructor
Xây lại Vector2d từ nhị phân tạo bởi `byte()`.
- `array.array` có `.frombytes`


In [11]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

# tag::VECTOR2D_V1[]
    @classmethod  # <1>
    def frombytes(cls, octets):  # <2>
        typecode = chr(octets[0])  # <3>
        memv = memoryview(octets[1:]).cast(typecode)  # <4>
        return cls(*memv)  # <5>
# end::VECTOR2D_V1[]

# 1: The `classmethod` decorator modifies a method so it can be called directly on a class.
# 2: No `self` argument, thay vào đó, class (Vector2d) được truyền với tên là `cls`

In [14]:
chr(octets[0])

'd'

In [15]:
octets[1:]

b'\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [16]:
memv = memoryview(octets[1:]).cast('d')
memv

In [13]:
v1_clone = Vector2d.frombytes(bytes(v1))
v1_clone

Vector2d(3.0, 4.0)

### classmethod vs staticmethod


Bất cứ ai đã học OO trong Java có thể thắc mắc tại sao Python có cả hai decorators này mà không chỉ một trong số chúng.

`classmethod`:
- Định nghĩa method để gọi trong class mà không phải tạo 1 instance
- Nhận class là tham số đầu tiên (cls)
- Dùng cho hàm tạo (constructors) thay thế

`staticmethod`:
- Method sẽ không nhận tham số đầu đặc biệt (cls)
- Như 1 hàm đơn giản, gọi ở class body thay vì ở module level
- Hiếm có usecases dùng

In [21]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args

Demo.klassmeth('spam') # Dù thế nào cũng nhận Demo class là tham số đầu tiên

(__main__.Demo, 'spam')

In [22]:
Demo.statmeth('spam')

('spam',)

## Formatted Displays

In [24]:
brl = 1 / 4.82
brl

0.20746887966804978

In [25]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.21 USD'

In [26]:
f'1 USD = {1 / brl:0.2f} BRL'

'1 USD = 4.82 BRL'

In [27]:
format(42, 'b')

'101010'

In [28]:
format(2 / 3, '0.1%') # tham số thứ 2 được gọi là format_spec

'66.7%'

In [29]:
format(v1)

'(3.0, 4.0)'

In [30]:
format(v1, '.2f')

TypeError: unsupported format string passed to Vector2d.__format__

--> Cần fix format ở class

In [32]:
# BEGIN VECTOR2D_V2
from array import array
import math


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        # 'p': custom format (polar coordinates: <r, θ>)
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
# END VECTOR2D_V2

In [33]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [34]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

In [35]:
'0.5fp'[:-1]

'0.5f'

## A Hashable Vector2d

Làm cho Vector2d có thể hashable --> khi đó, có thể build set/dict của Vectors

In [36]:
v1 = Vector2d(3, 4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [37]:
set([v1])

TypeError: unhashable type: 'Vector2d'

-->
- Implement `__hash__` và `__eq__` (đã có)
- Make vector instances immutable (Hash yêu cầu imutable)

In [39]:
v1.x, v1.y
v1.x = 7 # Hiện tại vẫn là mutable, cần đổi nó thành read only
v1.x

7

In [40]:
# Version 3:
from array import array
import math

class Vector2d:
    __match_args__ = ('x', 'y') # dùng ở phần Positional Pattern Matching

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x) # x --> __x: private attribute
        self.__y = float(y)

    @property # getter method
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        # Method cần đọc x, y vẫn như cũ. Nó sẽ đọc qua self.x thay vì private attribute
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash((self.x, self.y))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [41]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(1079245023883434373, 1994163070182233067)

In [42]:
{v1, v2}

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

In [43]:
set([v1,v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

## Supporting Positional Pattern Matching

To make `Vector2d` work with positional patterns, we need to add a class attribute named `__match_args__`

In [44]:
def positional_pattern_demo(v: Vector2d) -> None:
    match v:
        case Vector2d(0, 0):
            print(f'{v!r} is null')
        case Vector2d(0):
            print(f'{v!r} is vertical')
        case Vector2d(_, 0):
            print(f'{v!r} is horizontal')
        case Vector2d(x, y) if x==y:
            print(f'{v!r} is diagonal')
        case _:
            print(f'{v!r} is awesome')

positional_pattern_demo(v1)

Vector2d(3.0, 4.0) is awesome


`__match_args__` không cần bao gồm tất cả các thuộc tính đối tượng, dựa vào `__init__`:
- Thuộc tính bắt buộc thì điền zo
- Thuộc tính optional thì bỏ qua

## Complete Listing of Vector2d, Version 3

[vector2d_v3.py](https://github.com/huymq1710/example-code-2e/blob/master/11-pythonic-obj/vector2d_v3.py) là 1 ví dụ khi muốn có 1 full-fledged object.

NOTE

You should only implement these special methods if your application needs them.
- End users don’t care if the objects that make up the application are “Pythonic” or not.
- Pythonic quan trọng khi build 1 library/framework, nơi mà ta ko đoán được họ sẽ cần gì với nó.

## Private and “Protected” Attributes in Python

discuss the design and drawbacks of the private attribute self.__x

Ta không có `private` như Java. Ở Python, ta chỉ có cơ chế đơn giản để tránh overwriting 1 private attribute.

In [45]:
# Example 11-12. Private attribute names are “mangled” by prefixing the _ and the class name

v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [49]:
v1.x = 4

AttributeError: can't set attribute 'x'

Private names are mangled can read the private attribute directly

In [52]:
v1._Vector2d__x = 4 # <-- đây chính là cách để đổi giá trị ở Private attribute

In [51]:
v1.__dict__

{'_Vector2d__x': 4, '_Vector2d__y': 4.0}

*Name mangling* is about safety, not security: it’s designed to prevent accidental access

`__x__` là thiết bị an toàn, không phải thiết bị bảo mật: nó ngăn ngừa tai nạn chứ không phải phá hoại.
![image.png](image/11-1.png)

Bởi lẽ có thể đổi private attribute qua mangling như vậy, nhiều người thích dùng “protect” attributes (`self._x`)

"Never, ever use two leading underscores(`__x`)."


Quy ước:
- `_x`:  should not access such attributes from outside the class
- `ALL_CAPS`: Hằng số

Bởi lẽ vậy, đôi khi `self._x` được gọi là “private” / “protected”(not common) attribute.

## Saving Memory with __slots__

By default, Python stores the attributes of each instance in a `dict` named `__dict__`.

Nhưng khi định nghĩa `__slot__`:
- Python sẽ dùng nó thay cho `__dict__`
- Thuộc tính trong `__slot__` lưu ở 1 hidden array/tham chiếu --> tốn ít bộ nhớ hơn

In [56]:
class Pixel:
    __slots__ = ('x', 'y')

p = Pixel()
p.__dict__

AttributeError: 'Pixel' object has no attribute '__dict__'

In [57]:
p.color = 'red'

AttributeError: 'Pixel' object has no attribute 'color'

In [59]:
# Example 11-14. The OpenPixel is a subclass of Pixel

class OpenPixel(Pixel):
    pass

op = OpenPixel()
op.__dict__ # instances of OpenPixel have a __dict__

{}

In [62]:
# x có ở __slots__ --> x được lưu vào slot
op.x = 8
op.__dict__

{}

In [64]:
# color không có ở __slots__ --> color được lưu ở dict
op.color = 'green'
op.__dict__

{'color': 'green'}

If you want a subclass to have additional attributes, name them in `__slots__`

Another special per-instance attribute that you may want to keep is `__weakref__`

### Simple Measure of `__slot__` Savings

```
$ time python3 mem_test.py vector2d_v3
Selected Vector2d type: vector2d_v3.Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:      6,983,680
  Final RAM usage:  1,666,535,424

real	0m11.990s
user	0m10.861s
sys	0m0.978s
```


```
$ time python3 mem_test.py vector2d_v3_slots
Selected Vector2d type: vector2d_v3_slots.Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:      6,995,968
  Final RAM usage:    577,839,104

real	0m8.381s
user	0m8.006s
sys	0m0.352s
```

### Summarizing the Issues with `__slots__`

- Phải nhớ khai báo lại `__slots__` trong mỗi lớp con để ngăn các phiên bản của chúng có `__dict__`
- Instances chỉ có thể có các thuộc tính được liệt kê trong `__slots__`, trừ khi đưa `'__dict__'` vào `__slots__`
- Class dùng `__slots_` không thể dùng `@cached_property` decorator, rừ khi họ đặt tên rõ ràng `'__dict__'` trong `__slots__`.
- Instances không thể được mask là weak reference (Ch6 để gom rác), trừ khi thêm `'__weakref__'` vào `__slots__`.

## Overriding Class Attributes

In [65]:
Vector2d.typecode

'd'

Ví dụ 11-18. Tùy chỉnh một instance bằng cách đặt thuộc tính typecode được kế thừa từ lớp

In [66]:
from vector2d_v3 import Vector2d

v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [67]:
len(dumpd)

17

In [68]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [69]:
len(dumpf)

9

In [71]:
# Vector2d.typecode is unchanged; only the v1 instance uses typecode 'f'.
Vector2d.typecode

'd'